# Upper Animas PEST Pilot Point Setup

In [ ]:
import sys
import numpy as np
import pandas as pd
import flopy as flopy
import pyemu
import shapefile #the pyshp module
import os
import matplotlib.pyplot as plt
from apexmf import apexmf_pst_utils, apexmf_pst_par
import apexmf
apexmf.__version__

# 01. Set up and write apexmf.con file

In [ ]:
# working directory and file names
wd = "D:\\Workshops\\20220419_apexmf_opt\\APEX-MODFLOW"
rch_file = 'SITE75.RCH'

# calibration period
sim_start_day = '1/1/2002'
cali_start_day = '1/1/2002'
cali_end_day = '12/31/2002'
# time step
time_step = 'month'
# activate river parm
riv_parm = 'y'

# extract simulation (what our targets)
gw_level = 'y'
# locations
subs = [12, 57, 75]
grids = [5895, 6273]


# pilot points included
pps = ['hk0pp.dat', 'sy0pp.dat']
os.chdir(wd)

In [ ]:
con =  apexmf_pst_utils.create_apexmf_con(
                        wd, sim_start_day, cali_start_day, cali_end_day,
                        rch_file, subs,
                        riv_parm=riv_parm,
                        gw_level=gw_level,
                        grids=grids,
                        time_step=time_step,

                        pp_included=pps
                        )
con

In [ ]:
apexmf_pst_utils.init_setup(wd)

# 02. MODFLOW parameters with Pilot Points

In [ ]:
# m = flopy.modflow.Modflow.load(fs.MODEL_NAM,model_ws=wd,load_only=[]) #<-- load only prevents reading ibound
mfwd = os.path.join(wd, 'MODFLOW')
mname = 'mf_1000.nam'
m = flopy.modflow.Modflow.load(mname,
            model_ws=mfwd
            )
m.check()

In [ ]:
m.bas6.ibound[0].plot()

## Example of pp MODFLOW templates

In [ ]:
# Create pilot points as a shapefile
# we want hk pilot points in the top layer...
prefix_dict = {0:["hk0"]}
df_pp = pyemu.pp_utils.setup_pilotpoints_grid(ml=m,
                                              prefix_dict=prefix_dict,
                                              pp_dir=wd,
                                              tpl_dir=wd,
                                              every_n_cell=10,
                                              shapename='pp_hk.shp')
# pp_file = os.path.join(working_dir,"sypp.dat")
# assert os.path.exists(pp_file)

So cool, we now defined pilot points as a set of spatially distributed parameters...but how do go from pilot points to the model input HK array? Answer: geostatistics.  We need to calculate the geostatistical factors (weights) used to form the interpolated value for the HK value at each model cell - its a spatially-weighted combination of pilot point values

## Need to create Kriging factors and regularization inputs
Following the guidelines in _Approaches to Highly Parameterized Inversion: Pilot-Point Theory, Guidelines, and Research Directions_ https://pubs.usgs.gov/sir/2010/5168/

### First we need to define a couple geostatistical structures (e.g. variograms)

From _PEST Groundwater Data Utilities Part A: Overview_ page 43, there are 4 acceptable variogram types:

 1. *Spherical*  
### $\gamma\left(h\right)=c\times\left[1.5\frac{h}{a}-0.5\frac{h}{a}^3\right]$ if $h<a$
### $\gamma\left(h\right)=c$ if $h \ge a$  
     
 2. *Exponential*  
### $\gamma\left(h\right)=c\times\left[1-\exp\left(-\frac{h}{a}\right)\right]$  
     
 3. *Gaussian*  
### $\gamma\left(h\right)=c\times\left[1-\exp\left(-\frac{h^2}{a^2}\right)\right]$  
 
 4. *Power*  
### $\gamma\left(h\right)=c\times h^a$
     
 The number refers to `VARTYPE`. `BEARING` and `ANISOTROPY` only apply if there is a principal direction of anisotropy. $h$ is the separation distance, and $a$ is the range, expressed with the `A` parameter.


### First, let's create ``variogram`` and ``GeoStruct`` objects.  

These describe how HK varies spatailly, remember?

In [ ]:
v = pyemu.geostats.ExpVario(contribution=200,a=20100, bearing=0)
gs = pyemu.geostats.GeoStruct(variograms=v,nugget=0)
ax = gs.plot()
ax.grid()

In [ ]:
ok = pyemu.geostats.OrdinaryKrige(gs,df_pp)
df = ok.calc_factors_grid(m.sr,
                          var_filename= "hk0pp.var.ref",
#                           var_filename=pst_name.replace(".pst",".var.ref"),
#                           var_filename= ppf[:-3] + "var.ref",                          
                          minpts_interp=3,
                          maxpts_interp=30,
                          search_radius=200000.0,
                          verbose=True,
                          num_threads=12)

In [ ]:
# arr_var = np.loadtxt(pst_name.replace(".pst",".var.ref"))
arr_var = np.loadtxt("hk0pp.var.ref")
ax = plt.subplot(111,aspect="equal")
p = ax.imshow(arr_var,extent=m.sr.get_extent(),alpha=0.25)
plt.colorbar(p)
plt.tight_layout()
ax.scatter(df_pp.x,df_pp.y,marker='.',s=4,color='r')

In [ ]:
ppf = 'hk0pp.dat'

In [ ]:
ok.to_grid_factors_file(ppf+".fac")

In [ ]:
# generate random values
df_pp.loc[:,"parval1"] = np.random.random(df_pp.shape[0])
# save a pilot points file
pyemu.pp_utils.write_pp_file(ppf,df_pp)

In [ ]:
# interpolate the pilot point values to the grid
hk_arr = pyemu.utils.geostats.fac2real(ppf,factors_file=ppf+".fac",out_file=None)

In [ ]:
# plot
ax = plt.subplot(111,aspect='equal')
ax.imshow(hk_arr,interpolation="nearest",extent=m.sr.get_extent(),alpha=0.5)
ax.scatter(df_pp.x,df_pp.y,marker='.',s=4,color='k')

In [ ]:
#THE PREVIOUS CELLS ARE EXAMPLE, THIS CELL IS THE CONTINUATION OF SETUP CODE
lyrs = 1
cell_nums = [20]
hk_prefix = ['hk{}'.format(i) for i in range(lyrs)]
sy_prefix = ['sy{}'.format(i) for i in range(lyrs)]

In [ ]:
#LOOP CELL
#THIS CELL SHOULD BE REPEATED AS THE NUMBER OF MODFLOW PARAMTER WE WANT TO PREPARE
#HERE IS HAS sy_prefix, then we choose the correct contribtion and correct a, then hk and change contribution and a and so on
##ss_prefix contribution=2.4e-6;   sy_prefix contribution=0.8;   hk_prefix contribution=200
for s, c in zip(hk_prefix, cell_nums):
    prefix_dict = {0:[s]}
    df_pp = pyemu.pp_utils.setup_pilotpoints_grid(ml=m,
                                                  prefix_dict=prefix_dict,
                                                  pp_dir=wd,
                                                  tpl_dir=wd,
                                                  every_n_cell=c,
                                                  shapename='pp_{}.shp'.format(s))
    v = pyemu.geostats.ExpVario(contribution=200,a=20100)
    gs = pyemu.geostats.GeoStruct(variograms=v,nugget=0.0)
    ok = pyemu.geostats.OrdinaryKrige(gs,df_pp)
    df = ok.calc_factors_grid(
                m.sr,
                var_filename= "{}pp.var.ref".format(s),
                minpts_interp=3,
                maxpts_interp=30,
                search_radius=200000,
                verbose=True,
                num_threads=12)    
    ok.to_grid_factors_file("{}pp.dat.fac".format(s))

In [ ]:
# Create parm template file
sw_par = apexmf_pst_par.parm_to_tpl_file()
sw_par

# Create instruction file for observed depth to water

In [ ]:
os.chdir(wd)

In [ ]:
os.getcwd()

In [ ]:
mf_obs = pd.read_csv(
                    "MODFLOW/modflow.obs",
                    delim_whitespace=True,
                    skiprows = 2,
                    usecols = [3, 4],
                    index_col = 0,
                    names = ["grid_id", "mf_elev"],)
grid_ids = mf_obs.index.tolist()

In [ ]:
grid_ids

In [ ]:
apexmf_pst_utils.extract_depth_to_water(grids, sim_start_day, cali_end_day,)

In [ ]:
mfobd_file = 'dtw_day.obd'

In [ ]:
mfobd_df = pd.read_csv(
                    "MODFLOW/" + mfobd_file,
                    delim_whitespace=True,
                    index_col=0,
                    header=0,
                    parse_dates=True,
                    na_values=[-999, ""])

In [ ]:
apexmf_pst_utils.mf_obd_to_ins('dtw_5895.txt', 'gw_124', sim_start_day, cali_end_day)
apexmf_pst_utils.mf_obd_to_ins('dtw_6273.txt', 'gw_098', sim_start_day, cali_end_day)

In [ ]:
os.getcwd()

# Create instruction file for streamflow

In [ ]:
# file path
rch_file = 'SITE75.RCH'
# reach numbers that are used for calibration
subs = [12, 57 , 75]

In [ ]:
# extract month_streamflow
apexmf_pst_utils.extract_month_stf(rch_file, subs, sim_start_day, cali_start_day, cali_end_day)

In [ ]:
# because we have 3 streamgages let's loop for them
# read streamobd and get column names
stf_obd = pd.read_csv(
                    'stf_mon.obd',
                    sep='\t',
                    index_col=0,
                    parse_dates=True,
                    na_values=[-999, '']
                    )
obds = stf_obd.columns.tolist()
print(obds)
sim_files = ['stf_{:03d}.txt'.format(x) for x in subs]
print(sim_files)

In [ ]:
stf_obd

In [ ]:
apexmf_pst_utils.stf_obd_to_ins?

In [ ]:
# create instruction files for each sim file
for i in range(len(sim_files)):
    apexmf_pst_utils.stf_obd_to_ins(sim_files[i], obds[i], sim_start_day, cali_end_day, time_step='month')

# 03. Create a control file

## Create a dummy pst file 

In [ ]:
io_files = pyemu.helpers.parse_dir_for_io_files('.')
pst = pyemu.Pst.from_io_files(*io_files)
pyemu.helpers.pst_from_io_files(io_files[0], io_files[1], io_files[2], io_files[3], 'ani_dummy.pst')

# print(os.chdir(".."))
io_files

In [ ]:
# load the pre-constructed pst
# pst = pyemu.Pst(os.path.join(wd, 'ani_dummy.pst'))

In [ ]:
par = pst.parameter_data

In [ ]:
par

In [ ]:
for i in range(len(par)):
    if (par.iloc[i, 0][:2]) == 'sy':
        par.iloc[i, 6] = 'sy'
    elif par.iloc[i, 0][:2] == 'hk':
        par.iloc[i, 6] = 'hk'
    elif par.iloc[i, 0][:1] == 'p':
        par.iloc[i, 6] = 'apex'


In [ ]:
par.index.rename('parnme1', inplace=True)

In [ ]:
par = par.sort_values(by=['pargp', 'parnme'])
par

In [ ]:
for i in range(len(par)):
    if par.iloc[i, 6] == 'sy':
        par.iloc[i, 3] = 1.000000e-01 
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 0.800000e+00  
    elif par.iloc[i, 6] == 'hk':
        par.iloc[i, 3] = 1.000000e-01 
        par.iloc[i, 4] = 1.000000e-02
        par.iloc[i, 5] = 5.000000e+02

In [ ]:
par

In [ ]:
# APEX
pst.parameter_data = apexmf_pst_par.export_pardb_pest(par)

In [ ]:
par = pst.parameter_data
par

In [ ]:
obd = pst.observation_data
obd

In [ ]:
for i in range(len(obd)):
    if obd.iloc[i, 0][:2] == 'gw':
        obd.iloc[i, 3] = obd.iloc[i, 0][:6]
    else:
        obd.iloc[i, 3] = obd.iloc[i, 0][:7]

In [ ]:
print(obd)

## 2.3. Import measured data

In [ ]:
gwt_obd = pd.read_csv('MODFLOW/dtw_day.obd',
                       sep='\t',
                       index_col = 0,
                       parse_dates = True,
                       usecols=[0, 3, 4],
                       na_values=[-999, '']
                     )
gwt_obd = gwt_obd[sim_start_day:cali_end_day]
gwt_obd = gwt_obd.dropna(axis=1, how='all')

gwt_obd


In [ ]:
# gwt_obd = gwt_obd[gwtcols]
gwt_obd = gwt_obd.reindex(sorted(gwt_obd.columns), axis=1)
gwt_obd

In [ ]:
stf_obd = pd.read_csv(
    'stf_mon.obd',
    sep='\t',
    index_col = 0,
    parse_dates = True,
    na_values=[-999, '']
)
stf_obd = stf_obd[sim_start_day:cali_end_day]
stf_obd =  stf_obd.reindex(sorted(stf_obd.columns), axis=1)
stf_obd

In [ ]:
# Get sub list based on obd order
sub_order = []
for i in obd.obgnme.tolist():
    if i not in sub_order:
        sub_order.append(i)
sub_order

In [ ]:
# get total list from each sub obd, delete na vals
tot_obd = []
for i in sub_order[:2]:
    tot_obd += gwt_obd[i].dropna().tolist()
for j in sub_order[2:]:
    tot_obd += stf_obd[j].dropna().tolist()    
len(tot_obd)

In [ ]:
obd.loc[:, 'obsval'] = tot_obd
obd

# 4. Export control file

In [ ]:
pst.control_data.noptmax=0
pst.model_command = 'python forward_run.py'
pst.model_input_data = apexmf_pst_utils.modify_mf_tpl_path(pst.model_input_data)
pst.write('animas_pest.pst')